## Import Modules

In [1]:
import pandas as pd
import numpy as np

import torch
from ultralytics import settings
from ultralytics import YOLO

import os
import json

from pathlib import Path
import shutil
from tqdm.auto import tqdm
import yaml
from concurrent.futures import ThreadPoolExecutor, as_completed

import albumentations as A
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches






# Module Config

In [2]:
# Ultralytics settings
print(settings) 

# GPU Utilisation
print(torch.cuda.is_available())
device = torch.device("cuda")

{'settings_version': '0.0.4', 'datasets_dir': 'D:\\Tom\\GitHub\\Third Year Project\\datasets', 'weights_dir': 'D:\\Tom\\GitHub\\Third Year Project\\Intruder-Aircraft-Detection\\weights', 'runs_dir': 'D:\\Tom\\GitHub\\Third Year Project\\Intruder-Aircraft-Detection\\runs', 'uuid': 'fdb5c10788ffaa41a9047dc764dd8a0a3287d6bec12c0c66234ac745c0366efa', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}
True


## Load Dataset

In [3]:
# Base paths for the images and labels
train_images_path = 'datasets/AVOIDDS/images/train'
train_labels_path = 'datasets/AVOIDDS/labels/train'
val_images_path = 'datasets/AVOIDDS/images/valid'
val_labels_path = 'datasets/AVOIDDS/labels/valid'

# Load the metadata
metadata_path = 'datasets/AVOIDDS/metadata.json'
with open(metadata_path, 'r') as file:
    metadata = json.load(file)

# Function to create a DataFrame from images and labels
def create_dataframe(images_path, labels_path, metadata):

    # List all files in the directories
    image_files = [f for f in sorted(os.listdir(images_path)) if f.endswith('.jpg')]
    label_files = [f for f in sorted(os.listdir(labels_path)) if f.endswith('.txt')]
    
    # Create tempory DataFrame so that final dataframe is in correct order
    temp_df = pd.DataFrame({
        'image_path': [str(images_path + '/' + file) for file in image_files],
        'label_path': [str(labels_path + '/' + file) for file in label_files],
    })

    # Extract image indices to match with metadata
    df = pd.DataFrame()
    df['imageID'] = temp_df['image_path'].apply(lambda x: int(os.path.splitext(os.path.basename(x))[0]))

    # Add image and label paths to final dataframe
    df['image_path'] = temp_df['image_path']
    df['label_path'] = temp_df['label_path']
 
    # Add metadata to each image entry
    for key, value in metadata.items():
        if '.' in key:  # Key represents a range
            start, end = map(int, key.split('.'))
            df.loc[df['imageID'].between(start, end), 'metadata'] = json.dumps(value)

    # Convert the JSON strings in 'metadata' to dictionaries
    df['metadata'] = df['metadata'].apply(json.loads)

    # Expand the 'metadata' column into separate columns
    metadata_df = pd.json_normalize(df['metadata'])
    
    # Concatenate the expanded metadata back to the original DataFrame
    full_df = pd.concat([df.drop(['metadata'], axis=1), metadata_df], axis=1)

    return full_df

# Create the DataFrames for the train and validation sets
train_df = create_dataframe(train_images_path, train_labels_path, metadata)
valid_df = create_dataframe(val_images_path, val_labels_path, metadata)

#train_df.info()

# Create sub datasets

In [9]:
# removes all files and folders from dataset directory
def clear_directory(dir_path):    
    for item in os.listdir(dir_path):
        item_path = os.path.join(dir_path, item)
        if os.path.isfile(item_path) or os.path.islink(item_path):
            os.unlink(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)


## creates subsets of main dataset
def create_dataset(dataset_name, filtered_train_df, filtered_valid_df, class_names=['aircraft'], dataset_dir="datasets/"):
    new_dataset_dir = Path(dataset_dir) / dataset_name
    images_dir = Path(new_dataset_dir) / 'images'
    labels_dir = Path(new_dataset_dir) / 'labels'

    # Clear directories if they exist, to overwrite the dataset
    for subdir in ['train', 'valid']:
        img_subdir = images_dir / subdir
        label_subdir = labels_dir / subdir
        if img_subdir.exists():
            clear_directory(img_subdir)
        else:
            img_subdir.mkdir(parents=True, exist_ok=True)
        if label_subdir.exists():
            clear_directory(label_subdir)
        else:
            label_subdir.mkdir(parents=True, exist_ok=True)

    # Create directories
    for subdir in ['train', 'valid']:
        (images_dir / subdir).mkdir(parents=True, exist_ok=True)
        (labels_dir / subdir).mkdir(parents=True, exist_ok=True)

    def copy_file(src, dest):
        shutil.copy2(src, dest)

    def copy_files_concurrently(df, img_dest_dir, label_dest_dir):
        with ThreadPoolExecutor() as executor:
            # Prepare futures for image and label copying
            futures = [executor.submit(copy_file, row['image_path'], img_dest_dir / f"{Path(row['image_path']).name}") for _, row in df.iterrows()]
            futures += [executor.submit(copy_file, row['label_path'], label_dest_dir / f"{Path(row['label_path']).name}") for _, row in df.iterrows()]
            
            # Initialize progress bar
            pbar = tqdm(total=len(futures), desc='Copying files')
            for future in as_completed(futures):
                # Update progress bar upon task completion
                pbar.update(1)
            pbar.close()

    print("Copying training files:")
    copy_files_concurrently(filtered_train_df, images_dir / 'train', labels_dir / 'train')
    print("Copying validation files:")
    copy_files_concurrently(filtered_valid_df, images_dir / 'valid', labels_dir / 'valid')

    # Construct the YAML content with the desired structure
    yaml_content = {
        'path': str(f'../{new_dataset_dir}').replace('\\', '/'),  # Ensuring forward slashes
        'train': str('images/train'),
        'val': str('images/valid'),
        'names': {index: name for index, name in enumerate(class_names)}
    }  

    yaml_path = new_dataset_dir / f"{dataset_name}.yaml"
    with open(yaml_path, 'w') as file:
        yaml.dump(yaml_content, file, sort_keys=False)

    print(f"Dataset '{dataset_name}' created at {dataset_dir}")



In [10]:
# test dataset creation

#test_train_df = train_df[train_df['image_path'] == 'datasets/AVOIDDS/images/train/1.jpg']
#test_valid_df = valid_df[valid_df['image_path'] == 'datasets/AVOIDDS/images/valid/900.jpg']
#
#dataset_name = 'test'

test_train_df = train_df[train_df['ac'] == 'Cessna Skyhawk']
test_valid_df = valid_df[valid_df['ac'] == 'Cessna Skyhawk']

dataset_name = 'dataset'

create_dataset(dataset_name, test_train_df, test_valid_df, dataset_dir='datasets/cessna_only')

Copying training files:


Copying files:   0%|          | 0/43200 [00:00<?, ?it/s]

Copying validation files:


Copying files:   0%|          | 0/4800 [00:00<?, ?it/s]

Dataset 'dataset' created at datasets/cessna_only


# Augmentation Methods

In [12]:
# extracts class names and bboxes from all objects in label
def load_yolo_labels(label_path):
    with open(label_path, 'r') as file:
        labels = [line.strip().split() for line in file.readlines()]
        bboxes = [list(map(float, label[1:])) for label in labels]
        class_labels = [int(label[0]) for label in labels]
    return bboxes, class_labels

# takes class names and augmented bbox and converts into yolo label format
def format_yolo_label(class_labels, augmented_bboxes):
    label_str = ""
    for class_label, bbox in zip(class_labels, augmented_bboxes):
        label_str += f"{class_label} " + " ".join(f"{x:.6f}" for x in bbox) + "\n"
    return label_str

In [13]:
## horizontal/vertical flip

def augment_flip(image_path, label_path, orientation, p=1.0):
    # Load image - openCV
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Load labels
    bboxes, class_labels = load_yolo_labels(label_path)
    
    # Define the augmentation based on the orientation parameter
    if orientation == 'h':
        flip = A.HorizontalFlip(p=p)
    elif orientation == 'v':
        flip = A.VerticalFlip(p=p)
    else:
        raise ValueError("Orientation must be 'h' or 'v'")
    
    transform = A.Compose([
        flip,
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))
    
    # Apply transformation
    transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
    augmented_image = transformed['image']
    augmented_bboxes = transformed['bboxes']
    
    # YOLO formatted label: [class_id, x_center, y_center, width, height]
    augmented_label = format_yolo_label(class_labels, augmented_bboxes)
    
    return augmented_image, augmented_label

In [14]:
## rotation

def augment_rotation(image_path, label_path, angle, p=1.0):
   
    # Load image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Load labels
    bboxes, class_labels = load_yolo_labels(label_path)
    
    # Define the augmentation with rotation
    transform = A.Compose([
        A.Rotate(limit=(angle, angle), p=p, border_mode=cv2.BORDER_CONSTANT),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))
    
    # Apply transformation
    transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
    augmented_image = transformed['image']
    augmented_bboxes = transformed['bboxes']
    
    # Convert augmented bboxes and class labels back to YOLO format
    augmented_label = format_yolo_label(class_labels, augmented_bboxes)
    
    return augmented_image, augmented_label

In [15]:
## contrast and brightness

# Alpha - contrast control (1.0-3.0)
# Beta - brightness control (-100 to 100)
def augment_brightness_contrast(image_path, alpha=1.0, beta=0):
    
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Apply brightness and contrast adjustment
    augmented_image = np.clip(alpha * image.astype(np.float32) + beta, 0, 255).astype(np.uint8)
    
    return augmented_image

In [16]:
## histogram equalisation with CLAHE

def augment_histogram_equalization(image_path, p=1.0):
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Define the augmentation
    transform = A.Compose([
        A.CLAHE(p),
    ])
    
    # Apply the augmentation
    transformed = transform(image=image)
    augmented_image = transformed['image']
    
    return augmented_image

In [17]:
## white balancing - gray word algorithm

def augment_white_balance(image_path):
  
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Calculate the mean of each channel
    mr = np.mean(image[:, :, 0])
    mg = np.mean(image[:, :, 1])
    mb = np.mean(image[:, :, 2])
    
    # Calculate the overall mean
    mgray = (mr + mg + mb) / 3
    
    # Scale the channels based on the Gray World assumption
    image[:, :, 0] = np.clip(image[:, :, 0] * (mgray / mr), 0, 255)
    image[:, :, 1] = np.clip(image[:, :, 1] * (mgray / mg), 0, 255)
    image[:, :, 2] = np.clip(image[:, :, 2] * (mgray / mb), 0, 255)
    
    augmented_image = image.astype(np.uint8)
    return augmented_image

In [18]:
## sharpening 

def augment_sharpen(image_path):
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Kernels from literature research
    kernel_1 = np.array([[0, -1, 0],
                         [-1, 5, -1],
                         [0, -1, 0]], dtype=np.float32)
    
    kernel_2 = np.array([[-1, -2, -1],
                         [-2, 13, -2],
                         [-1, -2, -1]], dtype=np.float32)

    kernel_2 = np.array([[-1, -2, -1],
                         [-2, 16, -2],
                         [-1, -2, -1]], dtype=np.float32)
    
    # Apply the sharpening kernel to the image
    sharpened_image = cv2.filter2D(image, -1, kernel_1)
    
    return sharpened_image

In [19]:
## guassian noise

def augment_gaussian_noise(image_path, var_limit=(10.0, 50.0)):
   
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Define the augmentation
    transform = A.Compose([
        A.GaussNoise(var_limit=var_limit, mean=0, p=1.0),
    ])
    
    # Apply the augmentation
    transformed = transform(image=image)
    augmented_image = transformed['image']
    
    return augmented_image

In [20]:
## zoom in 

# zoom factor: No zoom = 1, Full zoom (bounding box takes up entire picture) = 10
def augment_zoom(image_path, label_path, zoom_factor=1.5):

    # adjust zoom factor so that 1 = no zoom
    zoom_factor = zoom_factor / 10

    # Load image and labels
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    bboxes, class_labels = load_yolo_labels(label_path)

    # Calculate the crop dimensions based on the bounding box and zoom factor
    x_center, y_center, bbox_width, bbox_height = bboxes[0]  # Assuming one object
    x_center *= image.shape[1]  # Convert from relative to absolute coordinates
    y_center *= image.shape[0]
    bbox_width *= image.shape[1]
    bbox_height *= image.shape[0]

    # Define the crop dimensions
    crop_width = int(bbox_width / zoom_factor)
    crop_height = int(bbox_height / zoom_factor)
    
    # Calculate the crop coordinates
    x_min = max(0, int(x_center - crop_width / 2))
    y_min = max(0, int(y_center - crop_height / 2))
    x_max = min(image.shape[1], int(x_center + crop_width / 2))
    y_max = min(image.shape[0], int(y_center + crop_height / 2))

    # Define Albumentations transform for cropping and resizing
    transform = A.Compose([
        A.Crop(x_min=x_min, y_min=y_min, x_max=x_max, y_max=y_max, p=1.0),
        A.Resize(height=image.shape[0], width=image.shape[1], p=1.0)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))
    
    # Apply transformation
    transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']
    
    # The transformed bboxes are already in YOLO format
    augmented_label = format_yolo_label(class_labels, transformed_bboxes)

    return transformed_image, augmented_label


In [23]:
## tester method to overlay bboxes on images

def overlay_bbox(image_path, label_path):

    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Load labels
    bboxes, class_labels = load_yolo_labels(label_path)
    
    fig, ax = plt.subplots(1)
    ax.imshow(image)
    
    height, width, _ = image.shape
    for bbox, class_label in zip(bboxes, class_labels):
        x_center, y_center, bbox_width, bbox_height = bbox
        x_min = (x_center - bbox_width / 2) * width
        y_min = (y_center - bbox_height / 2) * height
        
        rect = patches.Rectangle((x_min, y_min), bbox_width * width, bbox_height * height,
                                 linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.text(x_min, y_min - 2, str(class_label), color='red', fontsize=10, 
                 bbox=dict(facecolor='white', alpha=0.5, edgecolor='red', boxstyle='round'))
    
    plt.axis('off')
    plt.show()

In [21]:
#method to save image
def save_image(path, image):

    # Ensure the directory exists
    os.makedirs(os.path.dirname(path), exist_ok=True)
    

    # Convert from RGB to BGR
    image_to_save = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # Save the image
    cv2.imwrite(path, image_to_save)

In [22]:
# method to save label
def save_label(path, contents):
    # Ensure the directory exists
    os.makedirs(os.path.dirname(path), exist_ok=True)
    
    # Now, save the file
    with open(path, 'w') as label_file:
        label_file.write(contents)

In [ ]:
# block to test aug methods

image = '1'
dataset = 'train'
aug_type = 'flip'

test_image_path = f'datasets/test/images/{dataset}/{image}.jpg'
test_label_path = f'datasets/test/labels/{dataset}/{image}.txt'

aug_image_path = f'datasets/test-aug/images/{aug_type}/{image}.jpg'
aug_label_path = f'datasets/test-aug/labels/{aug_type}/{image}.txt'

aug_image, aug_label = augment_rotation(test_image_path, test_label_path, 45)

save_image(aug_image_path, aug_image)
save_label(aug_label_path, aug_label)

overlay_bbox(aug_image_path, aug_label_path)

In [ ]:
## create augmented datasets

def copy_file(src, dst):
    # Create the destination directory if it doesn't exist.
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    # Copy the file
    shutil.copy2(src, dst)

def copy_dataset_in_parallel(src, dst):
    # List all files in the original dataset directory and its subdirectories
    all_files = [os.path.join(dp, f) for dp, dn, filenames in os.walk(src) for f in filenames]
    # Calculate the relative paths
    relative_paths = [os.path.relpath(file, src) for file in all_files]
    
    # Create the destination directory
    os.makedirs(dst, exist_ok=True)
    
    # Start a progress bar
    pbar = tqdm(total=len(all_files), desc='Copying dataset')

    # Copy files in parallel
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        # Prepare the future copy operations
        futures = {executor.submit(copy_file, src_file, os.path.join(dst, rel_path)): rel_path for src_file, rel_path in zip(all_files, relative_paths)}
        # As each copy operation completes, update the progress bar
        for future in futures:
            future.result()  # Wait for each future to complete
            pbar.update(1)
    
    pbar.close()

def apply_augmentation(image_path, augmented_dataset_path, method_name, details):
    label_path = image_path.replace('images/train', 'labels/train').replace('.jpg', '.txt')
    imageID = os.path.basename(image_path)
    aug_image_filename = imageID.replace(".jpg", f"-{method_name}.jpg")
    aug_label_filename = imageID.replace(".jpg", f"-{method_name}.txt")
    
    # Determine which augmentation function to call based on the method name
    if method_name == 'flip':
        augmented_image, augmented_label = augment_flip(image_path, label_path, **details['parameters'])
    elif method_name == 'rotation':
        augmented_image, augmented_label = augment_rotation(image_path, label_path, **details['parameters'])
    elif method_name == 'brightness_contrast':
        augmented_image = augment_brightness_contrast(image_path, **details['parameters'])
        augmented_label = open(label_path).read()  # No change to label for this augmentation
    elif method_name == 'gaussian_noise':
        augmented_image = augment_gaussian_noise(image_path, **details['parameters'])
        augmented_label = open(label_path).read()  # No change to label for this augmentation
    elif method_name == 'histogram_equalization':
        augmented_image = augment_histogram_equalization(image_path, **details['parameters'])
        augmented_label = open(label_path).read()  # No change to label for this augmentation
    elif method_name == 'white_balance':
        augmented_image = augment_white_balance(image_path)
        augmented_label = open(label_path).read()  # No change to label for this augmentation
    elif method_name == 'sharpen':
        augmented_image = augment_sharpen(image_path)
        augmented_label = open(label_path).read()  # No change to label for this augmentation
    elif method_name == 'zoom':
        augmented_image, augmented_label = augment_zoom(image_path, label_path, **details['parameters'])
    
    # Save the augmented image and label
    save_image(augmented_dataset_path + '/images/train/' + aug_image_filename, augmented_image)
    save_label(augmented_dataset_path + '/labels/train/' + aug_label_filename, augmented_label)


def augment_dataset(original_dataset_path, augmentation_metadata):
   
    # Step 1: Copy the original dataset
    augmented_dataset_path = original_dataset_path + '_aug'
    if os.path.exists(augmented_dataset_path):
        shutil.rmtree(augmented_dataset_path)
    #shutil.copytree(original_dataset_path, augmented_dataset_path)
    copy_dataset_in_parallel(original_dataset_path, augmented_dataset_path)
    print(f"Copied dataset to: {augmented_dataset_path}")

    # Load the original training images
    train_images_dir = original_dataset_path + '/images/train'
    train_images = [os.path.join(train_images_dir, img_name) for img_name in os.listdir(train_images_dir)]


    # Prepare for augmentations    
    total_images = len(train_images)

    # Total augmentations for progress bar
    total_augmentations = 0
    for details in augmentation_metadata['methods'].values():
        total_augmentations += int(details['apply_to_percentage'] * total_images)
    pbar = tqdm(total=total_augmentations, desc="Augmenting Images and Labels")

    # Using ThreadPoolExecutor to parallelize image processing
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = []
        for method_name, details in augmentation_metadata['methods'].items():
            num_images_to_augment = int(details['apply_to_percentage'] * total_images)
            selected_images = np.random.choice(train_images, num_images_to_augment, replace=False)
            
            for image_path in selected_images:
                futures.append(executor.submit(apply_augmentation, image_path, augmented_dataset_path, method_name, details))
        
        # Wait for all futures to complete
        for future in futures:
            future.result()
            pbar.update(1)

    # Update the progress bar completion and close
    pbar.close()

    old_dataset_name = os.path.basename(original_dataset_path)

    yaml_path = os.path.join(augmented_dataset_path, f'{old_dataset_name}.yaml')
    with open(yaml_path, 'r') as file:
        yaml_data = yaml.safe_load(file)

    # Append '-augmented' to the dataset path in the YAML data
    yaml_data['path'] = yaml_data['path'] + '_aug'

    # Save the updated YAML data to a new file with '-augmented' appended to the filename
    augmented_yaml_path = yaml_path.replace('.yaml', '_aug.yaml')
    with open(augmented_yaml_path, 'w') as file:
        yaml.dump(yaml_data, file, default_flow_style=False, sort_keys=False)

    os.remove(yaml_path)

In [28]:
test_dataset_dir = "datasets/cessna_only/dataset"

#augmentation_metadata = {
#    'methods': {
#        'flip': {
#            'parameters': {
#                'orientation': 'h',  # Could be 'h' for horizontal or 'v' for vertical
#                'p': 1.0  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 1  # 50% of the training images
#        },
#        'rotation': {
#            'parameters': {
#                'angle': 45,  # Rotation angle
#                'p': 1.0  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 1  # 30% of the training images
#        },
#        'brightness_contrast': {
#            'parameters': {
#                'alpha': 1.5,  # Contrast control (1.0-3.0)
#                'beta': 50     # Brightness control (-100 to 100)
#            },
#            'apply_to_percentage': 0.2  # 20% of the training images
#        },
#        'gaussian_noise': {
#            'parameters': {
#                'var_limit': (10.0, 50.0),  # Variance range for noise
#            },
#            'apply_to_percentage': 1  # 10% of the training images
#        },
#        # Add other methods as needed
#    }
#}

augmentation_metadata = {
    'methods': {
        'flip': {
            'parameters': {
                'orientation': 'h',  # Could be 'h' for horizontal or 'v' for vertical
                'p': 1.0  # Probability of applying the augmentation
            },
            'apply_to_percentage': 0.1  # 50% of the training images
        }
    }
}

augment_dataset(test_dataset_dir, augmentation_metadata)

Copying dataset:   0%|          | 0/48001 [00:00<?, ?it/s]

Copied dataset to: datasets/cessna_only/dataset_aug


Augmenting Images and Labels:   0%|          | 0/2160 [00:00<?, ?it/s]

# Test Parameters

In [ ]:
# parameters for each specfic test














# Training

# Own Implementation of Object Detection System